In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [2]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


Wczytywanie Danych

In [8]:
cd "/content/drive/My Drive/Colab Notebooks/DataScienceMatrix/Matrix_two/DataScienceMatrix2"

/content/drive/My Drive/Colab Notebooks/DataScienceMatrix/Matrix_two/DataScienceMatrix2


In [10]:
df = pd.read_hdf('Data/car.h5')
df.shape

(106494, 155)

In [11]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [0]:
## Dummy Model

In [13]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [15]:
X = df[ ['car_id'] ].values
Y = df['car_id'].values
model = DummyRegressor()
model.fit(X,Y)
y_pred = model.predict(X)
mean_absolute_error(Y,y_pred)

40104.125306754606

In [16]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [18]:
df['price_currency'].value_counts(normalize = True) * 100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [19]:
df = df [ df['price_currency']!='EUR']
df.shape

(106290, 155)

In [25]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

## Features

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [32]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [35]:
X = df[cat_feats].values
Y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model,X,Y,cv = 3, scoring = 'neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [38]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,Y)

imp = PermutationImportance(m).fit(X,Y)
eli5.show_weights(imp,feature_names = cat_feats)

Weight,Feature
0.2551 ± 0.0093,param_napęd__cat
0.2005 ± 0.0013,param_faktura-vat__cat
0.1934 ± 0.0051,param_stan__cat
0.1444 ± 0.0093,param_rok-produkcji__cat
0.0641 ± 0.0032,param_moc__cat
0.0420 ± 0.0008,feature_kamera-cofania__cat
0.0414 ± 0.0023,param_skrzynia-biegów__cat
0.0275 ± 0.0017,param_marka-pojazdu__cat
0.0200 ± 0.0020,param_pojemność-skokowa__cat
0.0163 ± 0.0003,feature_bluetooth__cat
